In [1]:
pip install ultralytics

In [2]:
pip install git+https://github.com/ChaoningZhang/MobileSAM.git

  Cloning https://github.com/ChaoningZhang/MobileSAM.git to /tmp/pip-req-build-8n5v1cri
  Running command git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/MobileSAM.git /tmp/pip-req-build-8n5v1cri
  Resolved https://github.com/ChaoningZhang/MobileSAM.git to commit 01ea8d0f5590082f0c1ceb0a3e2272593f20154b
  Preparing metadata (setup.py) ... done


In [18]:
pip install segment_anything

In [3]:
pip install timm

In [4]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import requests
import time
from ultralytics import YOLO
# from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor



In [5]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [20]:
sam_checkpoint = "/content/drive/MyDrive/sam_vit_h_4b8939.pth"
# sam_checkpoint = "/content/drive/MyDrive/mobile_sam.pt"
model_type = "vit_h"
# model_type = "vit_t"
device="cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

In [7]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [8]:
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

In [9]:
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


In [10]:
def normalize_points(contour_points, image_width, image_height):
    # Normalize the points
    normalized_points = contour_points.astype(float) / np.array([image_width, image_height])
    return normalized_points

In [11]:
def convert_contour_to_yolov8(contour_points, class_index):
    # Flatten the contour points to a 1D array
    flattened_points = contour_points.reshape(-1, 2)

    # Create YOLOv8 label string
    label_string = f"{class_index}"
    for point in flattened_points:
        label_string += f" {point[0]} {point[1]}"
    return label_string

In [12]:
def download_image_from_drive(url, destination):
    response = requests.get(url)
    response.raise_for_status()
    with open(destination, 'wb') as file:
        file.write(response.content)
    print("Image downloaded successfully.")



In [13]:
def load_model(model_path):
    model=YOLO(model=model_path)
    return model

In [14]:
def extract_roi(image_path,coord_list):
    image=cv2.imread(image_path)
    im=image.copy()
    print("Coord List:", coord_list)
    x1, y1, x2, y2 = coord_list
    roi = im[y1:y2, x1:x2]
    # roi = image[y2:y1, x1:x2]
    return roi

In [15]:
def extract_labels(roi,model):

    img_dir="/content/drive/MyDrive/data/images/"
    img_list = os.listdir(img_dir)
    img_list.sort()
    image_path=img_dir + img_list[0]
    print(image_path)
    print(img_list[0])

    results=model.predict(source=roi,conf=0.5,save=True,show_labels=False)
    contours=[]
    # boxes=result.boxes
    for result in results:
        boxes = result.boxes

    i = 0
    label_string_list =[]
    while i < len(boxes.xyxy.tolist()):
        bbox = boxes.xyxy.tolist()[i]
        class_id = boxes.cls.tolist()[i]
        # print(bbox)

        # image = cv2.cvtColor(cv2.imread(roi), cv2.COLOR_BGR2RGB)
        predictor.set_image(roi)
        input_box = np.array(bbox)
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box,
            multimask_output=False,
        )
        # print(masks)
        mask = masks[0]
        # print(mask)
        contour, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours.extend(contour)

        contour_points = contours[i]
        class_index = int(class_id)
        image_height, image_width = roi.shape[:2]

        normalized_points = normalize_points(contour_points, image_width, image_height)
        label_string = convert_contour_to_yolov8(normalized_points, class_index)
        label_string_list.append(label_string)
        i += 1
    return label_string_list
    # output_path =image_path.split(".")[0] + '.txt'
    # print(output_path)
    # with open(output_path, 'w') as file:
    #     for item in label_string_list:
    #         file.write(str(item) + '\n')


In [22]:
start_time=time.time()
model_path="/content/drive/MyDrive/best_detect.pt"
model=load_model(model_path)

bounding_box_list=[[[1046, 298, 1152, 419],[720,217,862,320],[755,351,899,463],[689,85,832,184],[355,446,568,548],[583,403,754,492],[0,57,258,239],[545,254,715,328],[277,62,489,209],[888,184,1026,297],[15,278,290,436],[978,75,1112,165],[48,466,327,645],[920,314,1044,423],[1046,298,1152,419]],[[1689,1499,2384,1907],[2442,1488,3153,1906],[922,1498,1636,1914],[853,618,1599,1069],[2469,669,3249,983],[3551,1577,3838,1877]]]
fetch_dict={}
image_url_list = ["https://drive.google.com/uc?id=1RjaSIfNz11fcRvWgL1ER4ZWttxIZ8zzY","https://drive.google.com/uc?id=1TDsRtmbeou95BvQ1qK26wxc9tw9VXoSN"]

fetch_dict["url"]=[]
fetch_dict["coordinates"]=[]
fetch_dict["fullness"]=[]


for k in range(len(image_url_list)):
  destination_path = f"/content/drive/MyDrive/data/images/images{k}.jpg"
  fetch_dict["url"].append(destination_path.split("/")[-1])
  inst_bounding_box_list=bounding_box_list[k]
  fetch_dict["coordinates"].append(inst_bounding_box_list)

  download_image_from_drive(image_url_list[k], destination_path)

  fullness_list=[]
  for i in range(len(inst_bounding_box_list)):
    count_dict={}
    amb="None"

    roi=extract_roi(destination_path,inst_bounding_box_list[i])
    label_string_list=extract_labels(roi,model)
    output_path =destination_path.split(".")[0] + f'{i}.txt'
    print(output_path)
    with open(output_path, 'w') as file:
        for item in label_string_list:
              file.write(str(item) + '\n')
    file.close()
    with open(output_path, 'r') as file:
        lines = file.readlines()
        num_ones = 0
        num_zeros = 0
        for line in lines:
          line_list = line.split()
          class_label = int(line_list[0])
          if class_label == 1:
              num_ones += 1
          elif class_label == 0:
              num_zeros += 1
    print("Number of emptys:", num_ones)
    print("Number of objects:", num_zeros)
    count_dict[f"objects{i}"]=num_ones
    count_dict[f"emptys{i}"]=num_zeros
    if amb=="None":
      total_count=num_ones + num_zeros
    else:
      total_count=amb
    fullness=f"{round((num_ones/total_count)*100,2)}%"
    count_dict[f"fullness{i}"]=fullness
    # fetch_dict.update(count_dict)
    fullness_list.append(count_dict)
  fetch_dict["fullness"].append(fullness_list)

end_time=time.time()
print("Total Time :",end_time-start_time)
print(fetch_dict)






Image downloaded successfully.
Coord List: [1046, 298, 1152, 419]
/content/drive/MyDrive/data/images/images0.jpg
images0.jpg



0: 640x576 1 object, 18.7ms
Speed: 2.8ms preprocess, 18.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 576)
Results saved to runs/detect/predict8

0: 480x640 3 objects, 10.1ms
Speed: 3.4ms preprocess, 10.1ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict8


/content/drive/MyDrive/data/images/images00.txt
Number of emptys: 0
Number of objects: 1
Coord List: [720, 217, 862, 320]
/content/drive/MyDrive/data/images/images0.jpg
images0.jpg



0: 512x640 3 objects, 7.6ms
Speed: 2.1ms preprocess, 7.6ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict8


/content/drive/MyDrive/data/images/images01.txt
Number of emptys: 0
Number of objects: 3
Coord List: [755, 351, 899, 463]
/content/drive/MyDrive/data/images/images0.jpg
images0.jpg


...

0: 384x640 3 objects, 24.9ms
Speed: 2.8ms preprocess, 24.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 416x640 4 objects, 9.5ms
Speed: 4.0ms preprocess, 9.5ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict8


/content/drive/MyDrive/data/images/images12.txt
Number of emptys: 0
Number of objects: 3
Coord List: [853, 618, 1599, 1069]
/content/drive/MyDrive/data/images/images0.jpg
images0.jpg


/content/drive/MyDrive/data/images/images13.txt
Number of emptys: 0
Number of objects: 4
Coord List: [2469, 669, 3249, 983]
/content/drive/MyDrive/data/images/images0.jpg
images0.jpg


0: 288x640 3 objects, 14.4ms
Speed: 2.4ms preprocess, 14.4ms inference, 1.8ms postprocess per image at shape (1, 3, 288, 640)
Results saved to runs/detect/predict8

0: 640x640 1 object, 1 empty, 12.2ms
Speed: 3.8ms preprocess, 12.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict8


/content/drive/MyDrive/data/images/images14.txt
Number of emptys: 0
Number of objects: 3
Coord List: [3551, 1577, 3838, 1877]
/content/drive/MyDrive/data/images/images0.jpg
images0.jpg
/content/drive/MyDrive/data/images/images15.txt
Number of emptys: 1
Number of objects: 1
Total Time : 195.07569551467896
{'url': ['images0.jpg', 'images1.jpg'], 'coordinates': [[[1046, 298, 1152, 419], [720, 217, 862, 320], [755, 351, 899, 463], [689, 85, 832, 184], [355, 446, 568, 548], [583, 403, 754, 492], [0, 57, 258, 239], [545, 254, 715, 328], [277, 62, 489, 209], [888, 184, 1026, 297], [15, 278, 290, 436], [978, 75, 1112, 165], [48, 466, 327, 645], [920, 314, 1044, 423], [1046, 298, 1152, 419]], [[1689, 1499, 2384, 1907], [2442, 1488, 3153, 1906], [922, 1498, 1636, 1914], [853, 618, 1599, 1069], [2469, 669, 3249, 983], [3551, 1577, 3838, 1877]]], 'fullness': [[{'objects0': 0, 'emptys0': 1, 'fullness0': '0.0%'}, {'objects1': 0, 'emptys1': 3, 'fullness1': '0.0%'}, {'objects2': 0, 'emptys2': 3, 'full